In [1]:
import pandas as pd
import numpy as np
import itertools
from matplotlib import pyplot
# forecast monthly births with random forest
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.datasets import make_regression

In [2]:
series = read_csv('datacount7.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
train_end_date = '2023-07-01'
train_end_date = pd.to_datetime(train_end_date)
train = series.loc[series.index < train_end_date]
test = series.loc[series.index >= train_end_date]
trainvalues = train.values
trainvalues = trainvalues.reshape(-1, 1)
testvalues = test.values
testvalues = testvalues.reshape(-1, 1)

In [3]:
def series_to_supervised(data,n_in = 1,n_out = 1,dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
         cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        return agg.values

In [4]:
def search(train):
    # transform list into array
    train = asarray(train)
    # split into input and output columns
    trainX, trainy = train[:, :-1], train[:, -1]
    # fit model
    rf = RandomForestRegressor()  # 实例化随机森林模型
    param_grid = {
        'n_estimators': list(range(10, 141, 10)),  # 决策树数量
        'max_depth': list(range(2, 12, 1)),
        'min_samples_leaf' : list(range(3, 20, 1))
    }
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                               scoring='neg_mean_squared_error', cv=9)
    grid_search.fit(trainX, trainy)  # 使用训练集进行网格搜索
    print("Best parameters found: ", grid_search.best_params_)
    print("Lowest RMSE found: ", (-grid_search.best_score_) ** 0.5)

In [5]:
def search_with_dynamic_n_in(trainvalues, testvalues):
    for n_in in range(7, 31):
        traindata = series_to_supervised(trainvalues, n_in=n_in)
        testdata = series_to_supervised(testvalues, n_in=n_in)

        # Call the search function for each n_in
        search(traindata)
        print(n_in)

In [ ]:
search_with_dynamic_n_in(trainvalues,testvalues)

Best parameters found:  {'max_depth': 11, 'min_samples_leaf': 3, 'n_estimators': 60}
Lowest RMSE found:  107.91012160881161
7
